In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
newsgroups = fetch_20newsgroups(remove=('headers', 'footers'))

In [ ]:
newsgroups.target_names

In [ ]:
newsgroups.data[0]

In [ ]:
newsgroups.target_names[newsgroups.target[0]]

In [ ]:
origin = newsgroups.target_names[newsgroups.target[0]]
print(f"The post at index 0 first appeared in the '{origin}' group")

In [ ]:
len(newsgroups.data)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(stop_words='english')

In [ ]:
tf_matrix = vectorizer.fit_transform(newsgroups.data)

In [ ]:
type(tf_matrix)

In [ ]:
tf_np_matrix = tf_matrix.toarray()

In [ ]:
tf_vector = tf_np_matrix[0]
non_zero_indices = np.flatnonzero(tf_vector)
words = vectorizer.get_feature_names_out()
unique_words = [words[index] for index in non_zero_indices]
data = {'Word': unique_words,
'Count': tf_vector[non_zero_indices]}
df = pd.DataFrame(data).sort_values('Count', ascending=False)
print(f"After stop-word deletion, {df.shape[0]} unique words remain.")

In [ ]:
df.head(10)

In [ ]:
df.Word

In [ ]:
non_zero_indices

In [ ]:
sub_matrix = tf_np_matrix[:, non_zero_indices]

In [ ]:
sub_matrix[0]

In [ ]:
from sklearn.preprocessing import binarize

In [ ]:
binary_matrix = binarize(sub_matrix)

In [ ]:
binary_matrix

In [ ]:
binary_matrix.sum(axis=0).shape

In [ ]:
unique_post_mentions = binary_matrix.sum(axis=0)

In [ ]:
unique_post_mentions

In [ ]:
np_post_mentions = binarize(tf_np_matrix[:,non_zero_indices]).sum(axis=0)
csr_post_mentions = binarize(tf_matrix[:,non_zero_indices]).sum(axis=0)
print(f'NumPy matrix-generated counts:\n {np_post_mentions}\n')
print(f'CSR matrix-generated counts:\n {csr_post_mentions}')

In [ ]:
dataset_size = len(newsgroups.data)
document_frequencies = unique_post_mentions / dataset_size

In [ ]:
data = {'Word': unique_words,
'Count': tf_vector[non_zero_indices],
'Document Frequency': document_frequencies}

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
df_common_words = df[df['Document Frequency'] > 0.1]

In [ ]:
df_common_words.head(10)

In [ ]:
df_sorted = df.sort_values(['Count','Document Frequency'], ascending=[False, True])

In [ ]:
df_sorted.head(10)